In [166]:
from llama_index.legacy import (
    KeywordTableIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    download_loader,
    load_index_from_storage,
    VectorStoreIndex)

from llama_index.legacy.llms.vertex import Vertex
from langchain.chat_models import ChatVertexAI
from llama_index.legacy.llms.langchain import LangChainLLM
from langchain_google_vertexai import VertexAIEmbeddings
from llama_index.legacy.embeddings import LangchainEmbedding
from llama_index.legacy import Document
from llama_index.legacy.node_parser import SimpleNodeParser, HierarchicalNodeParser


from llama_index.legacy.vector_stores.chroma import ChromaVectorStore
import chromadb


from google.oauth2 import service_account
from llama_index.legacy import set_global_service_context
from google.cloud import aiplatform
import json 

## imports from og llama index file

In [167]:
from llama_index.legacy import VectorStoreIndex, SimpleDirectoryReader
from llama_index.legacy.embeddings import LangchainEmbedding
from llama_index.legacy.text_splitter import TokenTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from llama_index.legacy.llms.vertex import Vertex
from llama_index.legacy.node_parser import SimpleNodeParser, HierarchicalNodeParser
from llama_index.legacy import (
    KeywordTableIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    download_loader,
    load_index_from_storage,
    VectorStoreIndex)
from llama_index.legacy.retrievers import VectorIndexRetriever
from llama_index.legacy.prompts import (
    ChatMessage,
    ChatPromptTemplate,
    MessageRole,
    PromptTemplate,
)

from llama_index.legacy.postprocessor import NERPIINodePostprocessor, SentenceEmbeddingOptimizer
from llama_index.legacy import ServiceContext
from llama_index.legacy.schema import QueryBundle
from llama_index.legacy.schema import NodeWithScore, TextNode

from pathlib import Path
from tqdm.notebook import tqdm
from google.oauth2 import service_account

from llama_index.legacy import set_global_service_context
import re
import uuid
import os
from pathlib import Path
from pprint import pprint
import pandas as pd
import csv
from typing import List, Tuple, Dict
import time
import json

from llama_index.legacy.vector_stores.chroma import ChromaVectorStore
from llama_index.legacy import StorageContext
from llama_index.legacy.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb
from llama_index.core import Settings

from llama_index.legacy.llms.langchain import LangChainLLM
from langchain_google_vertexai import ChatVertexAI
from llama_index.legacy import Response
from llama_index.legacy.response_synthesizers import Refine
from llama_index.legacy.evaluation import SemanticSimilarityEvaluator
from llama_index.legacy.evaluation import RelevancyEvaluator
from llama_index.legacy.embeddings import SimilarityMode

import nest_asyncio
nest_asyncio.apply()

# additional
from llama_index.legacy.postprocessor import LongContextReorder
from llama_index.legacy.retrievers import AutoMergingRetriever
from llama_index.legacy.query_engine import TransformQueryEngine
from llama_index.legacy.query_engine import RetrieverQueryEngine
from llama_index.legacy.indices.query.query_transform.base import (
    HyDEQueryTransform,StepDecomposeQueryTransform
)
from llama_index.legacy.node_parser import get_leaf_nodes
from llama_index.legacy import Document
# llama_debug = LlamaDebugHandler(print_trace_on_end=True)
# callback_manager = CallbackManager([llama_debug])

## Initialize credentials, models, and embedding models

In [168]:
import vertexai
from vertexai.language_models import TextGenerationModel
from langchain_google_vertexai.llms import VertexAI

credentials = service_account.Credentials.from_service_account_file("credentials/vertex-test-417403-ce72ad032af7.json")
vertex_ai = Vertex(model="text-bison", project=credentials.project_id, location= "asia-southeast1", credentials=credentials, temperature=0.2)
text_model = TextGenerationModel.from_pretrained("text-bison@002")
text_vertex_ai = VertexAI(model_name="text-bison@002", project=credentials.project_id, location= "asia-southeast1", credentials=credentials, temperature=0.2)                       
chat_vertex_ai = ChatVertexAI(model_name="chat-bison@002", project=credentials.project_id, location= "asia-southeast1", credentials=credentials, temperature=0.2)
langchain_chat_vertex_ai = LangChainLLM(ChatVertexAI(model_name="chat-bison@002", project=credentials.project_id, location= "asia-southeast1", credentials=credentials, temperature=0.2))

selected_model = langchain_chat_vertex_ai

embed_model = LangchainEmbedding(VertexAIEmbeddings(model_name='textembedding-gecko-multilingual@latest'))

In [169]:
service_context = ServiceContext.from_defaults(llm=vertex_ai, embed_model=embed_model, chunk_size=1024, chunk_overlap=20)
set_global_service_context(service_context)
# need to set global service context becuase if not index = VectorStoreIndex.from_vector_store(vector_store, embed_model=Settings.embed_model) will produce an error
# it will say that it requires OPENAI key etc
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=3900, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='textembedding-gecko-multilingual@latest', embed_batch_size=10, callback_manager=<llama_index.legacy.callbacks.base.CallbackManager object at 0x000002D8EB24EA50>), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.legacy.callbacks.base.CallbackManager object at 0x000002D8EB24EA50>, id_func=<function default_id_func at 0x000002D8D4E38860>, chunk_size=1024, chunk_overlap=20, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.legacy.logger.base.LlamaLogger object at 0x000002D8EEA36D90>, callback_manager=<llama_index.legacy.callbacks.base.

In [170]:
import log_appointment as logapp # pingpings file

## {DONE} Produce csv storing query, model response (and soon others)

load query text files into one csv file

In [171]:
# import pandas as pd

# # Sample text strings (replace with your actual file paths)
# text_file1 = "../data/chief complaint.txt"
# text_file2 = "../data/making appointment.txt"
# text_file3 = "../data/medical experts.txt"

# # Create a list of text strings
# text_list = []
# for filename in [text_file1, text_file2, text_file3]:
#   with open(filename, "r") as f:
#     # Read all lines and concatenate them into a single string
#     text_content = f.readlines()  # Remove potential leading/trailing whitespace
#     text_list.extend(text_content)

# text_list = [line.replace('\n', '') for line in text_list]
# print(len(text_list))
# text_list

In [172]:
# # Create a DataFrame with a single column named "Text"
# # df = pd.DataFrame({"Text": text_list}, columns=["query"])
# df = pd.DataFrame(text_list, columns=["query"])

# filepath = "model_response.csv"
# df.to_csv(filepath, index=False)
# print(f"Data saved to {filepath}")

Create dataframe from model_response.csv file

In [173]:
# filepath = "model_response.csv"
# model_response_df = pd.read_csv(filepath)

Create qna dataframe from medical_experts_ans.csv (for appointments)

In [174]:
filepath = "medical_experts_ans.csv"
model_response_df = pd.read_csv(filepath)
model_response_df

,query,ref_answer
0,ไม่แน่ใจว่าจะเกี่ยวกับโรคหัวใจหรือไม่ครับ อายุ...,ทางเราแนะนำท่านเข้าพบแพทย์ในศูนย์หัวใจ โดยสามา...
1,เมื่อสองปีที่แล้วพี่สาวเป็นไทรอยเป็นพิษสุดท้าย...,ผู้ป่วยสามารถเข้ารับการวินิจฉัยเบื้องต้นที่ศูน...
2,ผมตื่นมามีอาการคล้ายสิวหัวช้างที่หลังหูสามเม็ด...,แนะนำให้ผู้ป่วยเข้าพบแพทย์เพื่อประเมินอาการโด...
3,อายุ 18 ปี ปวดปัสสาวะตอนกลางคืน แต่ไม่ได้ลุกไป...,ท่านสามารถเข้ารับการรักษาได้ที่ศูนย์ทางเดินปัส...
4,เวียนหัวไม่หมุน โคลงเคลง เพลีย มือเท้าเย็น ไปต...,ทางเราแนะนำให้รับการวินิจฉัยเบื้องต้นที่ศูนย์อ...
5,อยากหาแพทย์เฉพาะทางเพื่อรักษาอาการตกขาวออกมามา...,ผู้ป่วยสามารถเข้ารับการวินิจฉัยเบื้องต้นที่ศูน...
6,หายใจมีเสียงหวีด ไอ เหนื่อย มีเสมหะเเหนียว เป็...,ท่านสามารถเข้ารับการวินิจฉัยและรักษาได้ที่ศูนย...
7,มีผื่นคันเป็นขุยๆที่ขา แขนและลำตัว เวลาอากาศเย...,ทางโรงพยาบาลมีศูนย์โรคภูมิแพ้เฉพาะทางที่ท่านสา...
8,ปวดตาทั้งวัน เจอแสงจ้าก็ปวด และทำงานหน้าจอนาน ...,ทางเราแนะนำให้ท่านเข้าพบแพทย์ในศูนย์ตาเฉพาะทาง...
9,รู้สึกเหมือนมีอะไรเคลื่อนไหวในท้อง ตรงช่วงใต้ล...,ทางเราแนะนำให้รับการวินิจฉัยเบื้องต้นที่ศูนย์อ...


Create log_appointment.py log file

In [175]:
log_path = "../data/log_appointment.csv"
logapp.create_appoint_log(num_week=5, log_path=log_path, delete_existing=True)

## Import and load data
- How to create documents from a list of strings: 
https://docs.llamaindex.ai/en/stable/module_guides/loading/documents_and_nodes/usage_documents.html#defining-documents

### Load data from doctor_db.csv (modified from siriraj_doctor_details.csv for appointments) 

In [176]:
import pandas as pd

# unused
# reader = SimpleDirectoryReader(
#     input_files=["../data/siriraj_doctor_details.csv"]
# )
# csv_data = reader.load_data()
# print(f"Loaded {len(csv_data)} docs")
# csv_data

doctor_csv_file_path = "../data/doctor_db.csv"
doctor_csv_df = pd.read_csv(doctor_csv_file_path)
columns_to_drop = ['image_src', 'url', 'qualification']
doctor_csv_df = doctor_csv_df.drop(columns=[col for col in doctor_csv_df.columns if col in columns_to_drop])
doctor_csv_df

,name,table_check,expertise,doctorID
0,รศ. พญ.กติกา นวพันธุ์,"เสาร์ 14:00 - 17:00 น. สัปดาห์ที่ 1,2,3,4,5",เวชศาสตร์มารดาและทารกในครรภ์,0
1,รศ. พญ.กนกรัตน์ พรพาณิชย์,"พุธ 08:00 - 11:15 น. สัปดาห์ที่ 1,2,3,4,5",ศัลยกรรมกระดูกเบ้าตาและตกแต่ง,1
2,อ. พญ.กนกรัตน์ สุวรรณสิทธิ์,"พฤหัสบดี 13:30 - 17:00 น. สัปดาห์ที่ 1,2,3,4,5",โสต ประสาท การได้ยินและการทรงตัว,2
3,ทพญ.กนกอร ตันติพาณิชย์กูล,"พุธ 16:30 - 19:30 น. สัปดาห์ที่ 3,4",ทันตกรรมจัดฟัน การจัดฟันด้วยการผ่าตัดจัดฟัน,3
4,ผศ. นพ.กมล เผือกเพ็ชร,"พุธ 16:00 - 18:00 น. สัปดาห์ที่ 1,3,5 อาทิตย์ ...",กุมารเวชศาสตร์โลหิตวิทยาและมะเร็งในเด็ก,4
...,...,...,...,...
625,นพ.เอกราช ลิ่วธนมงคล,"เสาร์ 09:00 - 11:00 น. สัปดาห์ที่ 1,2,3,4,5 13...","โรคของลำไส้ใหญ่และทวารหนัก, ฝีคัณฑสูตร, ริดสีด...",625
626,รศ. นพ.เอกรินทร์ โชติกวาณิชย์,"พุธ 13:00 - 13:30 น. สัปดาห์ที่ 1,2,3,4,5 เสาร...","โรคนิ่วไต ท่อไต และกระเพาะปัสสาวะ, โรคเนื้องอก...",626
627,อ. นพ.เอกวิทย์ เกยุราพันธุ์,"ศุกร์ 09:00 - 11:00 น. สัปดาห์ที่ 1,2,3,4,5 อา...",ศัลยศาสตร์ออร์โธปิดิกส์,627
628,นพ.เอนกชัย ดกพฤกษ์,NaN,เวชศาสตร์ฉุกเฉิน,628


### Load data from clinic-list-final.csv


In [177]:
clinic_csv_file_path = "../data/clinic-list-final.csv"
clinic_csv_df = pd.read_csv(clinic_csv_file_path)
clinic_csv_df
# columns_to_drop = ['image_src', 'url', 'qualification']
# csv_df = csv_df.drop(columns=[col for col in csv_df.columns if col in columns_to_drop])
# csv_df

,office_hrs,location,phone,clinic_name,link
0,วันจันทร์-วันอาทิตย์เวลา 07:00-15:00 น.,ชั้น 4 โซน E,โทร:1474กด 1,คลินิกตรวจสุขภาพ,https://www.siphhospital.com/th/medical-servic...
1,วันจันทร์-วันอาทิตย์ตลอด 24 ชั่วโมง,ชั้น 1 โซน A,โทร:1474กด 1,คลินิกผู้ป่วยฉุกเฉิน,https://www.siphhospital.com/th/medical-servic...
2,วันจันทร์-วันอาทิตย์เวลา 08:00-16:00 น.,ชั้น 4 โซน C,โทร:1474กด 1,คลินิกฟื้นฟูหัวใจ,https://www.siphhospital.com/th/medical-servic...
3,วันจันทร์-วันอาทิตย์เวลา 07:00-21:00 น.,ชั้น 4 โซน D,โทร:1474กด 1,คลินิกเบาหวาน ไทรอยด์ และต่อมไร้ท่อ,https://www.siphhospital.com/th/medical-servic...
4,วันจันทร์-วันอาทิตย์เวลา 07:00-21:00 น.,ชั้น 4 โซน A,โทร:1474กด 1,ศูนย์ตา,https://www.siphhospital.com/th/medical-servic...
5,วันจันทร์-วันเสาร์เวลา 07:00-21:00 น.วันอาทิตย...,ชั้น 3 โซน A,โทร:1474กด 1,ศูนย์ทันตกรรม,https://www.siphhospital.com/th/medical-servic...
6,วันจันทร์-วันอาทิตย์เวลา 07:00-19:00 น.,ชั้น 4 โซน A,โทร:1474กด 1,ศูนย์ทางเดินปัสสาวะ,https://www.siphhospital.com/th/medical-servic...
7,วันจันทร์-วันศุกร์ เวลา 07:00-21:00 น.วันเสาร์...,ชั้น 4 โซน A,โทร:1474กด 1,ศูนย์ทางเดินอาหารและตับ,https://www.siphhospital.com/th/medical-servic...
8,วันจันทร์-วันอาทิตย์เวลา 07:00-21:00 น.,ชั้น 2 โซน E,โทร:1474กด 1,ศูนย์นรีเวช,https://www.siphhospital.com/th/medical-servic...
9,วันจันทร์-วันอาทิตย์เวลา 08:00-19:00 น.,ชั้น 3 โซน A,โทร:1474กด 1,ศูนย์ผิวหนังและศัลยกรรมตกแต่ง,https://www.siphhospital.com/th/medical-servic...


### {UNUSED} Load data from siriraj_doctor_details.csv (given)

Cleaning csv data

In [178]:
# import pandas as pd

# # unused
# # reader = SimpleDirectoryReader(
# #     input_files=["../data/siriraj_doctor_details.csv"]
# # )
# # csv_data = reader.load_data()
# # print(f"Loaded {len(csv_data)} docs")
# # csv_data

# csv_file_path = "../data/siriraj_doctor_details.csv"
# csv_df = pd.read_csv(csv_file_path)
# columns_to_drop = ['image_src', 'url', 'qualification']
# df = csv_df.drop(columns=[col for col in csv_df.columns if col in columns_to_drop])
# df

### TODO: Cluster expertise (not done) << FULL CODE IS IN MINNIELLM. IT IS REMOVED IN THIS FILE
- not sure if needed atm. will be used as a point of improvement later

In [179]:
clinic_csv_data = clinic_csv_df.apply(lambda x: ''.join(x.astype(str)), axis=1).tolist()
doctor_csv_data = doctor_csv_df.apply(lambda x: ''.join(x.astype(str)), axis=1).tolist()
clinic_csv_data = [Document(text=t) for t in clinic_csv_data] # produce Documents from csv data
doctor_csv_data = [Document(text=t) for t in doctor_csv_data] # produce Documents from csv data
print(len(clinic_csv_data))
print(clinic_csv_data)

22
[Document(id_='0d2f0c14-85f3-4f05-9e08-e049482a1194', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='วันจันทร์-วันอาทิตย์เวลา 07:00-15:00 น.ชั้น 4 โซน Eโทร:1474กด 1คลินิกตรวจสุขภาพhttps://www.siphhospital.com/th/medical-services/treatment-center/health-checkup-clinic', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='1ad666fa-b56a-4906-8577-5aab1e2aa0b8', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='วันจันทร์-วันอาทิตย์ตลอด 24 ชั่วโมงชั้น 1 โซน Aโทร:1474กด 1คลินิกผู้ป่วยฉุกเฉินhttps://www.siphhospital.com/th/medical-services/treatment-center/emergency-clinic', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='948dcd4b-d581-4

In [180]:
print(len(doctor_csv_data))
print(doctor_csv_data)

630
[Document(id_='183ba310-5018-4693-994d-06cc60645267', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='รศ. พญ.กติกา นวพันธุ์เสาร์ 14:00 - 17:00 น. สัปดาห์ที่ 1,2,3,4,5เวชศาสตร์มารดาและทารกในครรภ์0', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='d8a0f779-9e0c-40d4-ac97-8c297b176cbd', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='รศ. พญ.กนกรัตน์ พรพาณิชย์พุธ 08:00 - 11:15 น. สัปดาห์ที่ 1,2,3,4,5ศัลยกรรมกระดูกเบ้าตาและตกแต่ง1', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='c0989ca6-496b-494b-bc7c-d3ef0e620950', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, te

### Combine all loaded data into singular Document list docs

In [181]:
docs = []
docs.extend(clinic_csv_data)
# docs.extend(doctor_csv_data)


print(len(docs))
print(docs)

22
[Document(id_='0d2f0c14-85f3-4f05-9e08-e049482a1194', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='วันจันทร์-วันอาทิตย์เวลา 07:00-15:00 น.ชั้น 4 โซน Eโทร:1474กด 1คลินิกตรวจสุขภาพhttps://www.siphhospital.com/th/medical-services/treatment-center/health-checkup-clinic', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='1ad666fa-b56a-4906-8577-5aab1e2aa0b8', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='วันจันทร์-วันอาทิตย์ตลอด 24 ชั่วโมงชั้น 1 โซน Aโทร:1474กด 1คลินิกผู้ป่วยฉุกเฉินhttps://www.siphhospital.com/th/medical-services/treatment-center/emergency-clinic', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='948dcd4b-d581-4

### Create local ChromaDB Vectore Store (currently unused)

In [182]:
# # Create index and persist to Chroma vector store

# db = chromadb.PersistentClient(path="chroma_db")
# chroma_collection = db.get_or_create_collection("chroma_db")
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)

# index = VectorStoreIndex.from_documents(
#     documents = docs, storage_context=storage_context, embed_model=embed_model
# )

In [183]:
# Load index from persisted vector store

# currently index is used for retriever and not query engine

# db = chromadb.PersistentClient(path="chroma_db")
# chroma_collection = db.get_or_create_collection("chroma_db")
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# chromadb_index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

# chromadb_query_engine = chromadb_index.as_query_engine(similarity_top_k=2)
# vector_retriever = chromadb_index.as_retriever(similarity_top_k=2)

### Configuring query engine components

Hierarchical Node Parsing

In [184]:
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[1024, 512, 256]
)
nodes = node_parser.get_nodes_from_documents(docs)

leaf_nodes = get_leaf_nodes(nodes)
nodes_by_id = {node.node_id: node for node in nodes}

auto_merging_context = ServiceContext.from_defaults(
    llm=selected_model,
    embed_model= embed_model,
    node_parser=node_parser,
)

Store configured components in vector store called merging_index

In [185]:
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

# automerging_index = VectorStoreIndex(leaf_nodes, storage_context=storage_context, service_context=auto_merging_context)
clinic_index = VectorStoreIndex(leaf_nodes, storage_context=storage_context, service_context=auto_merging_context)

# TODO: fix the code to store in chromadb
# currently chromadb is not used

# merging_index_dir = "./merging_index"
# siriraj_doctor_index_dir = "./siriraj_doctor_index"
# clinic_doctor_index_dir = "./clinic_doctor_index"
clinic_index_dir = "./clinic_index"


clinic_index.storage_context.persist(persist_dir=clinic_index_dir)

Load automerging index

In [186]:
# merging_index_dir = "./merging_index"
# siriraj_doctor_index_dir = "./siriraj_doctor_index"
# clinic_doctor_index_dir = "./clinic_doctor_index"
clinic_index_dir = "./clinic_index"

# loaded_storage_context = StorageContext.from_defaults(persist_dir=merging_index_dir) # load the existing index
loaded_storage_context = StorageContext.from_defaults(persist_dir=clinic_index_dir) # load the existing index
# automerging_index = load_index_from_storage(loaded_storage_context)
clinic_index = load_index_from_storage(loaded_storage_context)


In [187]:
# automerging_as_retriever = automerging_index.as_retriever(similarity_top_k=10)
# automerging_retriever = AutoMergingRetriever(
#     automerging_as_retriever, 
#     automerging_index.storage_context, 
#     verbose=True
# )

automerging_as_retriever = clinic_index.as_retriever(similarity_top_k=10)
automerging_retriever = AutoMergingRetriever(
    automerging_as_retriever, 
    clinic_index.storage_context, 
    verbose=True
)

### Query engine

In [188]:
reorder = LongContextReorder()
hyde = HyDEQueryTransform(llm=vertex_ai, include_original=True)
retriever_query_engine = RetrieverQueryEngine.from_args(automerging_retriever,
                                              node_postprocessors=[reorder],
                                              )
transform_query_engine = TransformQueryEngine(retriever_query_engine, query_transform=hyde)

TESTING: Response production

In [189]:
query = "ถ้ามีอาการเจ็บหน้าอกและหอบหืดขณะออกกำลังกายควรปรึกษาแพทย์ด้านใด"
response=transform_query_engine.query(query)
print("response: ", response.response)
# print("source nodes: ", response.source_nodes[0])
# print("source node text: ", response.source_nodes[0].node.text)
# print("number of source nodes: ", len(response.source_node))

> Merging 1 nodes into parent node.
> Parent node id: ad8e8e14-33c2-47db-85e0-88756c1671eb.
> Parent node text: ศูนย์โรคไต บริการฟอกเลือดวันจันทร์-วันเสาร์  เวลา 06:00-21:00 น.ศูนย์อายุรกรรมโรคไตและทางเดินปัสส...

> Merging 1 nodes into parent node.
> Parent node id: 12660726-f4f7-4a4f-b97f-3ecf756464d9.
> Parent node text: ทุกวัน ตลอด 24 ชั่วโมง(ผู้ป่วยนัดหมาย 07:00-21:00 น.)ชั้น 1 โซน Dโทร:1474กด 1ศูนย์เอกซเรย์https:/...

> Merging 1 nodes into parent node.
> Parent node id: 00f02560-6104-4875-8760-94e79fd7f08b.
> Parent node text: วันจันทร์-วันอาทิตย์เวลา 07:00 - 19:00 น.ชั้น 4 โซน Cโทร:1474กด 1ศูนย์หัวใจhttps://www.siphhospit...

> Merging 1 nodes into parent node.
> Parent node id: 279a1cde-df5b-4424-9292-9e4aab7a083c.
> Parent node text: วันจันทร์-วันอาทิตย์เวลา 07:00-21:00 น.ชั้น 3 โซน Dโทร:1474กด 1ศูนย์หู คอ จมูกhttps://www.siphhos...

> Merging 1 nodes into parent node.
> Parent node id: 7ceb5a66-2dc6-4fa3-9dda-7e26adcb2de7.
> Parent node text: วันจันทร์-วันอาทิตย์เวลา 07:00-2

Inspecting response and context nodes

In [190]:
response.response

' ศูนย์หัวใจ'

In [191]:
print(response.source_nodes[0])

Node ID: 1c39c5b2-0164-4c3e-9e5a-e55ba1a6b894
Text: ทุกวัน ตลอด 24 ชั่วโมง(ผู้ป่วยนัดหมาย 07:00-21:00 น.)ชั้น 1 โซน
Dโทร:1474กด 1ศูนย์เอกซเรย์https://www.siphhospital.com/th/medical-
services/treatment-center/x-ray-center
Score:  0.655



In [192]:
response.source_nodes[0].node.text

'ทุกวัน ตลอด 24 ชั่วโมง(ผู้ป่วยนัดหมาย 07:00-21:00 น.)ชั้น 1 โซน Dโทร:1474กด 1ศูนย์เอกซเรย์https://www.siphhospital.com/th/medical-services/treatment-center/x-ray-center'

In [193]:
len(response.source_nodes)

10

Context Transformations - PII Example
- adding context to model response generation

In [194]:
pii_processor = NERPIINodePostprocessor()

def filter_pii_fn(**kwargs):
    # run optimizer
    query_bundle = QueryBundle(query_str=kwargs["query_str"])

    new_nodes = pii_processor.postprocess_nodes(
        [NodeWithScore(node=TextNode(text=kwargs["context_str"]))],
        query_bundle=query_bundle,
    )
    new_node = new_nodes[0]
    return new_node.get_content()

In [195]:
qa_prompt_tmpl_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query.\n"
    "If you need to state yes or no, state it in Thai only\n"
    "If the question is a yes/no question, answer either yes or no in Thai before giving your reasoning for the answer.\n"
    "The reasoning should contain the relevant information and be concise if possible. Don't reiterate the same information twice.\n"
    "If the question asks about the date and time for an apppointment, answer in the following format only.\n"
    "(day) (time) (full name of doctor) (doctor's expertise)\n"
    "for example: วันจันทร์ เวลา 09:00 - 13:00 น. doctor: รศ. พญ.กติกา นวพันธุ์ expertise: เวชศาสตร์ฟื้นฟู \n" 
    "Also, the format of the answer should be as similar to the format in the context information as possible. This includes numbering order and indentation. \n"
    "The answer should not include the name of the document where the information is gotten from.\n"
    # "However, if the context information does not contain the answer for the query, answer เอกสารไม่มีระบุไว้\n"
    "Query: {query_str}\n"
    "Answer: "
)

qa_prompt_tmpl_str2 = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query.\n"
    "If the query has date time, "
    "If you need to state yes or no, state it in Thai only\n"
    "If the question is a yes/no question, answer either yes or no in Thai before giving your reasoning for the answer.\n"
    "The reasoning should contain the relevant information and be concise if possible. Don't reiterate the same information twice.\n"
    "If the question asks about the date and time for an apppointment, answer in the following format only.\n"
    "{day_time} {full_name} ({expertise})\n"
    "for example: วันจันทร์ เวลา 09:00 - 13:00 น. doctor: รศ. พญ.กติกา นวพันธุ์ expertise: เวชศาสตร์ฟื้นฟู \n" 
    "Also, the format of the answer should be as similar to the format in the context information as possible. This includes numbering order and indentation. \n"
    "The answer should not include the name of the document where the information is gotten from.\n"
    "However, if the context information does not contain the answer for the query, answer เอกสารไม่มีระบุไว้\n"
    "Query: {query_str}\n"
    "Answer: "
    "1. {day_time} {full_name} ({expertise})\n"
    "2. {day_time} {full_name} ({expertise})\n"
    "..."
)

In [196]:
import llama_index.legacy.response_synthesizers

qa_prompt_tmpl = PromptTemplate(
    qa_prompt_tmpl_str2, function_mappings={"context_str": filter_pii_fn}
)


prompts_dict = transform_query_engine.get_prompts()
print(list(prompts_dict.keys()))

transform_query_engine.update_prompts(
    {"query_engine:response_synthesizers": qa_prompt_tmpl}
) # use merging index


['query_transform:hyde_prompt', 'query_engine:response_synthesizer:text_qa_template', 'query_engine:response_synthesizer:refine_template']


### Produce dataframe containing query, model response, and context

In [197]:
def produce_response(query):
    # response = chromadb_query_engine.query(query)
    response = transform_query_engine.query(query)
    return response

def produce_context(query):
    # retrieved_nodes = vector_retriever.retrieve(query)
    retrieved_nodes = automerging_retriever.retrieve(query)
    context_str = "\n\n".join([n.get_content() for n in retrieved_nodes])
    return context_str

In [198]:
model_response_df

,query,ref_answer
0,ไม่แน่ใจว่าจะเกี่ยวกับโรคหัวใจหรือไม่ครับ อายุ...,ทางเราแนะนำท่านเข้าพบแพทย์ในศูนย์หัวใจ โดยสามา...
1,เมื่อสองปีที่แล้วพี่สาวเป็นไทรอยเป็นพิษสุดท้าย...,ผู้ป่วยสามารถเข้ารับการวินิจฉัยเบื้องต้นที่ศูน...
2,ผมตื่นมามีอาการคล้ายสิวหัวช้างที่หลังหูสามเม็ด...,แนะนำให้ผู้ป่วยเข้าพบแพทย์เพื่อประเมินอาการโด...
3,อายุ 18 ปี ปวดปัสสาวะตอนกลางคืน แต่ไม่ได้ลุกไป...,ท่านสามารถเข้ารับการรักษาได้ที่ศูนย์ทางเดินปัส...
4,เวียนหัวไม่หมุน โคลงเคลง เพลีย มือเท้าเย็น ไปต...,ทางเราแนะนำให้รับการวินิจฉัยเบื้องต้นที่ศูนย์อ...
5,อยากหาแพทย์เฉพาะทางเพื่อรักษาอาการตกขาวออกมามา...,ผู้ป่วยสามารถเข้ารับการวินิจฉัยเบื้องต้นที่ศูน...
6,หายใจมีเสียงหวีด ไอ เหนื่อย มีเสมหะเเหนียว เป็...,ท่านสามารถเข้ารับการวินิจฉัยและรักษาได้ที่ศูนย...
7,มีผื่นคันเป็นขุยๆที่ขา แขนและลำตัว เวลาอากาศเย...,ทางโรงพยาบาลมีศูนย์โรคภูมิแพ้เฉพาะทางที่ท่านสา...
8,ปวดตาทั้งวัน เจอแสงจ้าก็ปวด และทำงานหน้าจอนาน ...,ทางเราแนะนำให้ท่านเข้าพบแพทย์ในศูนย์ตาเฉพาะทาง...
9,รู้สึกเหมือนมีอะไรเคลื่อนไหวในท้อง ตรงช่วงใต้ล...,ทางเราแนะนำให้รับการวินิจฉัยเบื้องต้นที่ศูนย์อ...


Produce model response for all queries

In [199]:
model_response_df['model_response'] = model_response_df['query'].apply(produce_response)
time.sleep(5)
model_response_df['context'] = model_response_df['query'].apply(produce_context)

> Merging 1 nodes into parent node.
> Parent node id: 00f02560-6104-4875-8760-94e79fd7f08b.
> Parent node text: วันจันทร์-วันอาทิตย์เวลา 07:00 - 19:00 น.ชั้น 4 โซน Cโทร:1474กด 1ศูนย์หัวใจhttps://www.siphhospit...

> Merging 1 nodes into parent node.
> Parent node id: ad8e8e14-33c2-47db-85e0-88756c1671eb.
> Parent node text: ศูนย์โรคไต บริการฟอกเลือดวันจันทร์-วันเสาร์  เวลา 06:00-21:00 น.ศูนย์อายุรกรรมโรคไตและทางเดินปัสส...

> Merging 1 nodes into parent node.
> Parent node id: 12660726-f4f7-4a4f-b97f-3ecf756464d9.
> Parent node text: ทุกวัน ตลอด 24 ชั่วโมง(ผู้ป่วยนัดหมาย 07:00-21:00 น.)ชั้น 1 โซน Dโทร:1474กด 1ศูนย์เอกซเรย์https:/...

> Merging 1 nodes into parent node.
> Parent node id: 353f60d6-b228-4141-b501-60a66e2280e6.
> Parent node text: วันจันทร์-วันอาทิตย์เวลา 08:00-16:00 น.ชั้น 4 โซน Cโทร:1474กด 1คลินิกฟื้นฟูหัวใจhttps://www.siphh...

> Merging 1 nodes into parent node.
> Parent node id: fb0ea514-50f9-46df-bf61-5f3f682c2c56.
> Parent node text: วันจันทร์-วันอาทิตย์เวลา 07:00-2

> Merging 1 nodes into parent node.
> Parent node id: ad8e8e14-33c2-47db-85e0-88756c1671eb.
> Parent node text: ศูนย์โรคไต บริการฟอกเลือดวันจันทร์-วันเสาร์  เวลา 06:00-21:00 น.ศูนย์อายุรกรรมโรคไตและทางเดินปัสส...

> Merging 1 nodes into parent node.
> Parent node id: fb0ea514-50f9-46df-bf61-5f3f682c2c56.
> Parent node text: วันจันทร์-วันอาทิตย์เวลา 07:00-21:00 น.ชั้น 4 โซน Dโทร:1474กด 1คลินิกเบาหวาน ไทรอยด์ และต่อมไร้ท่...

> Merging 1 nodes into parent node.
> Parent node id: 12660726-f4f7-4a4f-b97f-3ecf756464d9.
> Parent node text: ทุกวัน ตลอด 24 ชั่วโมง(ผู้ป่วยนัดหมาย 07:00-21:00 น.)ชั้น 1 โซน Dโทร:1474กด 1ศูนย์เอกซเรย์https:/...

> Merging 1 nodes into parent node.
> Parent node id: 6e13f1b5-47c0-4d63-8495-823e104b0072.
> Parent node text: วันจันทร์-วันอาทิตย์เวลา 07:00-21:00 น.ชั้น 2 โซน Dโทร:1474กด 1ศูนย์อายุรกรรมhttps://www.siphhosp...

> Merging 1 nodes into parent node.
> Parent node id: 5a2ea640-0e98-4afe-be9b-d884468070f5.
> Parent node text: วันจันทร์-วันอาทิตย์เวลา 08:00 –

In [200]:
model_response_df

,query,ref_answer,model_response,context
0,ไม่แน่ใจว่าจะเกี่ยวกับโรคหัวใจหรือไม่ครับ อายุ...,ทางเราแนะนำท่านเข้าพบแพทย์ในศูนย์หัวใจ โดยสามา...,จากอาการที่คุณเล่ามา น่าจะเป็นอาการของโรคหัวใ...,ศูนย์โรคไต บริการฟอกเลือดวันจันทร์-วันเสาร์ เ...
1,เมื่อสองปีที่แล้วพี่สาวเป็นไทรอยเป็นพิษสุดท้าย...,ผู้ป่วยสามารถเข้ารับการวินิจฉัยเบื้องต้นที่ศูน...,จากอาการที่เล่ามา พี่สาวของคุณอาจมีความเสี่ยง...,ศูนย์โรคไต บริการฟอกเลือดวันจันทร์-วันเสาร์ เ...
2,ผมตื่นมามีอาการคล้ายสิวหัวช้างที่หลังหูสามเม็ด...,แนะนำให้ผู้ป่วยเข้าพบแพทย์เพื่อประเมินอาการโด...,ศูนย์หู คอ จมูก,ทุกวัน ตลอด 24 ชั่วโมง(ผู้ป่วยนัดหมาย 07:00-21...
3,อายุ 18 ปี ปวดปัสสาวะตอนกลางคืน แต่ไม่ได้ลุกไป...,ท่านสามารถเข้ารับการรักษาได้ที่ศูนย์ทางเดินปัส...,จากอาการที่เล่ามา คล้ายกับอาการของโรคกระเพาะป...,วันจันทร์-วันอาทิตย์เวลา 07:00-19:00 น.ชั้น 4 ...
4,เวียนหัวไม่หมุน โคลงเคลง เพลีย มือเท้าเย็น ไปต...,ทางเราแนะนำให้รับการวินิจฉัยเบื้องต้นที่ศูนย์อ...,ศูนย์หัวใจ,ทุกวัน ตลอด 24 ชั่วโมง(ผู้ป่วยนัดหมาย 07:00-21...
5,อยากหาแพทย์เฉพาะทางเพื่อรักษาอาการตกขาวออกมามา...,ผู้ป่วยสามารถเข้ารับการวินิจฉัยเบื้องต้นที่ศูน...,จากอาการที่คุณเล่ามา น่าจะเป็นอาการของโรคทางเ...,วันจันทร์-วันอาทิตย์เวลา 07:00-19:00 น.ชั้น 4 ...
6,หายใจมีเสียงหวีด ไอ เหนื่อย มีเสมหะเเหนียว เป็...,ท่านสามารถเข้ารับการวินิจฉัยและรักษาได้ที่ศูนย...,จากอาการที่คุณเล่ามา แนะนำให้ไปพบแพทย์ที่ศูนย...,วันจันทร์-วันอาทิตย์เวลา 07:00-21:00 น.ชั้น 3 ...
7,มีผื่นคันเป็นขุยๆที่ขา แขนและลำตัว เวลาอากาศเย...,ทางโรงพยาบาลมีศูนย์โรคภูมิแพ้เฉพาะทางที่ท่านสา...,ศูนย์โรคภูมิแพ้ ชั้น 3 โซน D โทร 1474 กด 1,วันจันทร์-วันอาทิตย์เวลา 08:00 – 20:00 น.ชั้น ...
8,ปวดตาทั้งวัน เจอแสงจ้าก็ปวด และทำงานหน้าจอนาน ...,ทางเราแนะนำให้ท่านเข้าพบแพทย์ในศูนย์ตาเฉพาะทาง...,จากอาการที่เล่ามา น่าจะเป็นอาการของโรคตาแห้ง ...,วันจันทร์-วันอาทิตย์เวลา 07:00-21:00 น.ชั้น 4 ...
9,รู้สึกเหมือนมีอะไรเคลื่อนไหวในท้อง ตรงช่วงใต้ล...,ทางเราแนะนำให้รับการวินิจฉัยเบื้องต้นที่ศูนย์อ...,ศูนย์ทางเดินอาหารและตับ,ทุกวัน ตลอด 24 ชั่วโมง(ผู้ป่วยนัดหมาย 07:00-21...


record to csv

In [201]:
fp = "mod04_test_clinics_only.csv"
model_response_df.to_csv(fp, index=False, mode='w')  # Set index=False to exclude the index column
print("CSV file saved:", fp)

CSV file saved: mod04_test_clinics_only.csv


### Testing response production

In [202]:
# query1 = "ถ้ามีอาการเจ็บหน้าอกและหอบหืดขณะออกกำลังกายควรปรึกษาแพทย์ด้านใด"
# query2 = "ดิฉันต้องการนัดพบนักจิตวิทยาเพื่อการปรึกษาเรื่องความเครียด สะดวกมาพบเฉพาะสัปดาห์แรกและสัปดาห์สุดท้ายของเดือน สามารถมาหาหมอได้วันไหนบ้าง"
# query3 = "อยากหาแพทย์เฉพาะทางเพื่อรักษาอาการตกขาวออกมามากปนกับปัสสาวะ มีปัสสาวะแสบและปวดท้องน้อย"
appointment_q = "ช่วยจัดสรรเวลาเพื่อการปรึกษาเรื่องการเตรียมตัวก่อนการผ่าตัดหัวใจให้หน่อย ผมต้องการทราบวันที่ว่างของแพทย์เพื่อจัดเตรียมตารางเวลาของผมให้เหมาะสม"

response = produce_response(appointment_q)


> Merging 1 nodes into parent node.
> Parent node id: 353f60d6-b228-4141-b501-60a66e2280e6.
> Parent node text: วันจันทร์-วันอาทิตย์เวลา 08:00-16:00 น.ชั้น 4 โซน Cโทร:1474กด 1คลินิกฟื้นฟูหัวใจhttps://www.siphh...

> Merging 1 nodes into parent node.
> Parent node id: 00f02560-6104-4875-8760-94e79fd7f08b.
> Parent node text: วันจันทร์-วันอาทิตย์เวลา 07:00 - 19:00 น.ชั้น 4 โซน Cโทร:1474กด 1ศูนย์หัวใจhttps://www.siphhospit...

> Merging 1 nodes into parent node.
> Parent node id: 12660726-f4f7-4a4f-b97f-3ecf756464d9.
> Parent node text: ทุกวัน ตลอด 24 ชั่วโมง(ผู้ป่วยนัดหมาย 07:00-21:00 น.)ชั้น 1 โซน Dโทร:1474กด 1ศูนย์เอกซเรย์https:/...

> Merging 1 nodes into parent node.
> Parent node id: ad8e8e14-33c2-47db-85e0-88756c1671eb.
> Parent node text: ศูนย์โรคไต บริการฟอกเลือดวันจันทร์-วันเสาร์  เวลา 06:00-21:00 น.ศูนย์อายุรกรรมโรคไตและทางเดินปัสส...

> Merging 1 nodes into parent node.
> Parent node id: fb0ea514-50f9-46df-bf61-5f3f682c2c56.
> Parent node text: วันจันทร์-วันอาทิตย์เวลา 07:00-2

In [203]:
print("response:" , response)

response:  ข้อมูลที่คุณให้มาไม่มีตารางเวลาสำหรับการปรึกษาเรื่องการเตรียมตัวก่อนการผ่าตัดหัวใจ ดังนั้นจึงไม่สามารถจัดสรรเวลาให้ได้


In [204]:
context =  str(produce_context(query))
print("context: " , context)

> Merging 1 nodes into parent node.
> Parent node id: 12660726-f4f7-4a4f-b97f-3ecf756464d9.
> Parent node text: ทุกวัน ตลอด 24 ชั่วโมง(ผู้ป่วยนัดหมาย 07:00-21:00 น.)ชั้น 1 โซน Dโทร:1474กด 1ศูนย์เอกซเรย์https:/...

> Merging 1 nodes into parent node.
> Parent node id: 279a1cde-df5b-4424-9292-9e4aab7a083c.
> Parent node text: วันจันทร์-วันอาทิตย์เวลา 07:00-21:00 น.ชั้น 3 โซน Dโทร:1474กด 1ศูนย์หู คอ จมูกhttps://www.siphhos...

> Merging 1 nodes into parent node.
> Parent node id: ad8e8e14-33c2-47db-85e0-88756c1671eb.
> Parent node text: ศูนย์โรคไต บริการฟอกเลือดวันจันทร์-วันเสาร์  เวลา 06:00-21:00 น.ศูนย์อายุรกรรมโรคไตและทางเดินปัสส...

> Merging 1 nodes into parent node.
> Parent node id: 7ceb5a66-2dc6-4fa3-9dda-7e26adcb2de7.
> Parent node text: วันจันทร์-วันอาทิตย์เวลา 07:00-21:00 น.ชั้น 3 โซน Cโทร:1474กด 1ศูนย์ระบบการหายใจhttps://www.siphh...

> Merging 1 nodes into parent node.
> Parent node id: 5a2ea640-0e98-4afe-be9b-d884468070f5.
> Parent node text: วันจันทร์-วันอาทิตย์เวลา 08:00 –

record to csv

In [205]:
import pandas as pd

data = [query, response, context]
df = pd.DataFrame([data], columns=["query", "response", "context"])

fp = "test_results.csv"
df.to_csv(fp, mode='a', header=False, index=False)
print(f"Data saved to {fp}")

Data saved to test_results.csv
